In [1]:
!pip install flask
!pip install pyserial
!pip install opencv-python
!pip install nbimporter
!pip install torch torchaudio torchvision
!jupyter nbconvert --to script text_processing.ipynb
!jupyter nbconvert --to script camera_operation.ipynb
!jupyter nbconvert --to script braille_translate.ipynb

[NbConvertApp] Converting notebook text_processing.ipynb to script
[NbConvertApp] Writing 5212 bytes to text_processing.py
[NbConvertApp] Converting notebook camera_operation.ipynb to script
[NbConvertApp] Writing 675 bytes to camera_operation.py
[NbConvertApp] Converting notebook braille_translate.ipynb to script
[NbConvertApp] Writing 8351 bytes to braille_translate.py


In [8]:
from flask import Flask, render_template, Response, request, jsonify
import serial
import time
import os
import cv2
import threading
import io
from PIL import Image
from camera_operation import taking_picture
from text_processing import text_correction, text_detection
from braille_translate import text_to_braille

app = Flask(__name__)

# 연결할 시리얼 포트와 baudrate 설정
ser = serial.Serial('COM7', 9600)

# 변수 초기화
camera = cv2.VideoCapture(1)
camera.set(cv2.CAP_PROP_FRAME_WIDTH, 1080) 
camera.set(cv2.CAP_PROP_FRAME_HEIGHT, 1080)
recognized_texts_global = []  # 인식된 텍스트 리스트
current_text_index = -1  # 리스트 위치 변수
braille_texts = []
labelled_img_path = None
word = ""
last_arduino_message = ""  # Arduino로부터 받은 마지막 메시지를 저장하는 변수
current_status_message = "촬영 버튼을 누르기 전 책을 화면과 같이 왼쪽 페이지와 오른쪽 페이지를 구분할 수 있도록 배치하십시오."

def read_from_port(ser):
    global recognized_texts_global, current_text_index, braille_texts, labelled_img_path, word, last_arduino_message, current_status_message
    
    while True:
        reading = ser.readline().decode().strip()
        current_status_message = ""
        
        print("===============  Received from Arduino -> ", reading, "  ===============")
        if reading == "Take Picture":
            # 메시지 업데이트
            last_arduino_message = "촬영 버튼"
            
            # 1 - 이미지 촬영
            print("< 1 - 이미지 촬영 >")
            img_path = taking_picture(camera)

            # 널값인지 검사
            if img_path is None:
                current_status_message = "이미지 촬영에 실패했습니다. 카메라를 확인하시고 다시 촬영해주세요."
                continue
            else:
                current_status_message = "이미지 촬영이 완료됐습니다. 텍스트 인식을 진행합니다."
            
            # 2 - 텍스트 인식
            print("< 2 - 텍스트 인식 >")
            recognized_texts, labelled_img_path = text_detection(img_path)

            # 인식된 텍스트 처리
            words_list = [word for text in recognized_texts for word in text.split()]
            try:
                corrected_texts = text_correction(words_list)
            except ValueError as e:
                print(f"예외 발생: {e}")
                continue
            
            filtered_texts = [text for text in corrected_texts if text.strip()]
            print("인식된 텍스트 >>> ", filtered_texts)
            recognized_texts_global = filtered_texts  # 새로운 텍스트로 업데이트
            current_text_index = -1  # 새로운 텍스트 리스트로 업데이트되면 인덱스 초기화
            
            # 3 - 텍스트를 점자로 변환하여 저장
            print("< 3 - 텍스트 점자로 변환 >")
            braille_texts = text_to_braille(recognized_texts_global)
            print("점자 텍스트 >>> ", braille_texts)
            current_status_message = "점자 변환이 완료됐습니다. 다음 단어 출력 버튼과 이전 버튼 출력을 눌러 점자 확인이 가능합니다."
            
        elif reading == "Next":
            last_arduino_message = "다음 단어 출력 버튼"
            next_text()
            
        elif reading == "Previous":
            last_arduino_message = "이전 단어 출력 버튼"
            previous_text()

def next_text():
    global current_text_index, word, current_status_message, recognized_texts_global, braille_texts
    
    if len(recognized_texts_global) == 0:
        current_status_message = "리스트가 비어있습니다. 촬영부터 진행해주세요."
        print("[Warning] 리스트가 비어있습니다.")
        
    elif current_text_index < len(recognized_texts_global) - 1:
        current_text_index += 1
        word = recognized_texts_global[current_text_index]
        send_text_to_arduino(word, braille_texts[current_text_index])
    else:
        current_status_message = "인식된 텍스트를 다 읽었습니다."
        print("리스트의 끝점입니다.")

def previous_text():
    global current_text_index, word, current_status_message, recognized_texts_global, braille_texts
    
    if len(recognized_texts_global) == 0:
        current_status_message = "리스트가 비어있습니다. 촬영부터 진행해주세요."
        print("[Warning] 리스트가 비어있습니다.")
        
    elif current_text_index > 0:
        current_text_index -= 1
        word = recognized_texts_global[current_text_index]
        send_text_to_arduino(word, braille_texts[current_text_index])
        
    else:
        current_status_message = "첫 번째 텍스트이므로 그 이전 단어가 없습니다."
        print("리스트의 시작점입니다.")

def send_text_to_arduino(text, braille):
    print("단어:", text)
    print("점자:", braille)
    ser.write((braille + "\n").encode())

def generate_frames():
    while True:
        success, frame = camera.read()  # 카메라로부터 프레임 읽기
        if not success:
            break
        else:
            # 프레임 크기 가져오기
            height, width, _ = frame.shape

            # 중앙선 그리기
            center_x = width // 2
            cv2.line(frame, (center_x, 0), (center_x, height), (0, 255, 0), 2)

            # 텍스트 출력
            cv2.putText(frame, 'Left Page', (50, height // 2), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
            cv2.putText(frame, 'Right Page', (center_x + 270, height // 2), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

            # JPEG 형식으로 인코딩
            ret, buffer = cv2.imencode('.jpg', frame)
            frame = buffer.tobytes()

            yield (b'--frame\r\n'
                   b'Content-Type: image/jpeg\r\n\r\n' + frame + b'\r\n')

@app.route('/')
def index():
    return render_template('kdn_index.html')

@app.route('/video')
def video():
    return Response(generate_frames(), mimetype='multipart/x-mixed-replace; boundary=frame')

@app.route('/labelled_image')
def labelled_image():
    img = cv2.imread(labelled_img_path)
    img_pil = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    img_io = io.BytesIO()
    img_pil.save(img_io, 'JPEG', quality=70)
    img_io.seek(0)
    return Response(img_io.getvalue(), mimetype='image/jpeg')

@app.route('/arduino_received_message')
def arduino_received_message():
    # Arduino로부터 받은 마지막 메시지를 반환합니다.
    return jsonify({"message": last_arduino_message})

@app.route('/send_to_arduino')
def send_to_arduino():
    # Arduino로 보낸 문자열 반환
    return jsonify({"message": word})

@app.route('/current_status')
def get_current_status():
    # 현재 진행 상황을 출력
    return jsonify({"message": current_status_message})

@app.route('/braille_output')
def braille_output():
    # 현재 인식된 단어의 점자를 반환
    if current_text_index >= 0 and current_text_index < len(braille_texts):
        braille_number = braille_texts[current_text_index]  # 점자 숫자를 가져옴
        
        # 점자 숫자에 맞는 이미지 파일 경로 생성
        image_file = f"C:\\Users\\peter\\capstone\\Braille Picture\\braille_{braille_number}.png"
        
        # 파일이 존재하는지 확인
        if os.path.exists(image_file):
            return jsonify({"braille_image": image_file})
        else:
            return jsonify({"braille_image": "점자 이미지가 존재하지 않습니다."})
    else:
        return jsonify({"braille_image": "점자가 없습니다."})

if __name__ == '__main__':
    thread = threading.Thread(target=read_from_port, args=(ser,))
    thread.start()
    app.run(debug=False, host='0.0.0.0', port=5000) 

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.0.31:5000
Press CTRL+C to quit
127.0.0.1 - - [23/Nov/2024 12:19:03] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [23/Nov/2024 12:19:05] "GET /video?rand=0.33844264452525885 HTTP/1.1" 200 -
127.0.0.1 - - [23/Nov/2024 12:19:05] "GET /video?rand=0.28228066746151304 HTTP/1.1" 200 -
127.0.0.1 - - [23/Nov/2024 12:19:05] "GET /video?rand=0.4597393774144174 HTTP/1.1" 200 -
127.0.0.1 - - [23/Nov/2024 12:19:05] "GET /video?rand=0.7359823592045738 HTTP/1.1" 200 -
127.0.0.1 - - [23/Nov/2024 12:19:05] "GET /video?rand=0.037766307802062915 HTTP/1.1" 200 -
127.0.0.1 - - [23/Nov/2024 12:19:05] "GET /video?rand=0.2678874662783919 HTTP/1.1" 200 -
127.0.0.1 - - [23/Nov/2024 12:19:05] "GET /video?rand=0.45458391000456255 HTTP/1.1" 200 -
127.0.0.1 - - [23/Nov/2024 12:19:05] "GET /video?rand=0.6206435776143642 HTTP/1.1" 200 -
127.0.0.1 - - [23/Nov/2024 12:19:05] "GET /video?rand=0.8662987013673715 HTTP/1.1" 200 -


===============  Received from Arduino ->     ===============
